<a href="https://colab.research.google.com/github/Lucas-Buk/IMT/blob/main/Cancer_GMM_%2B_TSNE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Bibliotecas e instalações**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pickle

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# **Dicionário de variáveis**

*   **ESCOLARI**: Código para escolaridade do paciente (int = 1).

      1 – ANALFABETO

      2 – ENS. FUND. INCOMPLETO

      3 – ENS. FUND. COMPLETO

      4 – ENSINO MÉDIO

      5 – SUPERIOR

      9 – IGNORADA
*   **IDADE**: Idade do paciente (int = 3).
*   **SEXO**: Sexo do paciente (int = 1). 

      1 – MASCULINO

      2 – FEMININO
*   **UFNASC**: UF de nascimento (char = 2). Outras opções: SI - Sem informação; OP - Outro país.	
*   **UFRESID**: UF de residência (char = 2). Outras opções: OP - Outro país.
*   **IBGE**: Código da cidade de residência do paciente segundo IBGE com digito verificador (char = 7).
*   **CIDADE**: Cidade de residência do paciente (char = 200).
*   **CATEATEND**: Categoria de atendimento ao diagnóstico (int = 1). 

      1 - CONVENIO

      2 - SUS

      3 – PARTICULAR

      9 – SEM INFORMAÇÃO
*   **DTCONSULT**: Data da 1ª consulta (date = 10). Formato: DD/MM/YYYY	
*   **CLINICA**: Código da clinica (int = 2).

      1 – ALERGIA/IMUNOLOGIA

      2 – CIRURGIA CARDIACA

      3 – CIRURGIA CABEÇA E PESCOÇO

      4 – CIRURGIA GERAL

      5 – CIRURGIA PEDIATRICA

      6 – CIRURGIA PLASTICA

      7 – CIRURGIA TORAXICA

      8 – CIRURGIA VASCULAR

      9 – CLINICA MEDICA

      10 – DERMATOLOGIA

      11 – ENDOCRINOLOGIA

      12 – GASTROCIRURGIA
      
      13 – GASTROENTEROLOGIA
      
      14 – GERIATRIA
      
      15 – GINECOLOGIA
      
      16 – GINECOLOGIA / OBSTETRICIA
      
      17 – HEMATOLOGIA
      
      18 – INFECTOLOGIA
      
      19 – NEFROLOGIA
      
      20 – NEUROCIRURGIA
      
      21 – NEUROLOGIA
      
      22 – OFTALMOLOGIA
      
      23 – ONCOLOGIA CIRURGICA
      
      24 – ONCOLOGIA CLINICA
      
      25 – ONCOLOGIA PEDIATRICA
      
      26 – ORTOPEDIA
      
      27 – OTORRINOLARINGOLOGIA
      
      28 – PEDIATRIA
      
      29 – PNEUMOLOGIA
      
      30 – PROCTOLOGIA
      
      31 – RADIOTERAPIA
      
      32 – UROLOGIA
      
      33 – MASTOLOGIA
      
      34 – ONCOLOGIA CUTANEA
      
      35 – CIRURGIA PELVICA
      
      36 – CIRURGIA ABDOMINAL
      
      37 – ODONTOLOGIA
      
      38 – TRANSPLANTE HEPATICO
      
      99 – IGNORADO	
*   **DIAGPREV**: Diagnóstico e tratamento anterior (int = 1).

      1 – SEM DIAGNÓSTICO / SEM TRATAMENTO
      
      2 – COM DIAGNÓSTICO / SEM TRATAMENTO
      
      3 – COM DIAGNÓSTICO / COM TRATAMENTO
      
      4 – OUTROS	
*   **DTDIAG**: Data do diagnóstico (date = 10). Formato: DD/MM/YYYY	
*   **BASEDIAG**: Código da base do diagnóstico (int = 1).
      
      1 – EXAME CLINICO
      
      2 – RECURSOS AUXILIARES NÃO MICROSCÓPICOS
      
      3 – CONFIRMAÇÃO MICROSCÓPICA
      
      4 – SEM INFORMAÇÃO	
*   **TOPO**: Código da topografia (char = 4). Formato: C999 	
*   **TOPOGRUP**: Grupo da topografia	(char = 3). Formato: C99	
*   **DESCTOPO**: Descrição da Topografia (char = 80).
*   **MORFO**: Código da morfologia (char = 5). Formato: 99999
*   **DESCMORFO**: Descrição da morfologia (char = 80).
*   **EC**: Estádio clínico (char = 5).
*   **ECGRUP**: Grupo do estadiamento clínico (char = 3).

      0 - Tumores primários, classificados como in situ

      I - Tumores localizados

      II - Tumores com envolvimento regional por extensão direta

      III - Tumores com envolvimento regional de linfonodos
      
      IV - Tumores com metástase à distância 

      X - Para tumores não avaliados pelo profissional responsável ou sem informação sobre estadiamento anotada no prontuário

      Y - Para tumores em que não se aplica a classificação TNM. São os tumores não sólidos (por exemplo, as leucemias)
*   **T**: Classificação TNM - T (char = 5).
*   **N**: Classificação TNM - N (char = 5).	
*   **M**: Classificação TNM - M (char = 3).	
*   **PT**: Estadiamento pós cirúrgico (char = 5).
*   **PN**: Estadiamento pós cirúrgico (char = 5).	
*   **PM**: Estadiamento pós cirúrgico (char = 3).	
*   **S**: Classificação TNM - S (int = 1). Domínio: 0; 1; 2; 3; 8 – NÃO SE APLICA; 9 – X
*   **G**: Classificação TNM – G (Grau) (char = 5). 

      Domínio (exceto C40, C41, C381, C382, C383, C47, C48 e C49):
      0; 1; 2; 3; 4; 8 – NÃO SE APLICA; 9 – X

      Domínio (somente C40, C41, C381, C382, C383, C47, C48 e C49):
      ALTO; BAIXO; 8 – NÃO SE APLICA; 9 – X
	
*   **LOCALTNM**: Classificação TNM - Localização (int = 1).
      
      1 – SUPERIOR
      
      2 – MEDIO
      
      3 – INFERIOR
      
      8 – NÃO SE APLICA
      
      9 – X	
*   **IDMITOTIC**: Classificação TNM – Índice Mitótico (int = 1).
      
      1 – ALTA
      
      2 – BAIXA
      
      8 – NÃO SE APLICA
      
      9 – X	
*   **PSA**: Classificação TNM - PSA (int = 1).
      
      1 – MENOR QUE 10
      
      2 – MAIOR OU IGUAL A 10 E MENOR QUE 20
      
      3 – MAIOR OU IGUAL A 20
      
      8 – NÃO SE APLICA
      
      9 – X	
*   **GLEASON**: Classificação TNM - Gleason (int = 1).
      
      1 – MENOR OU IGUAL A 6
      
      2 – IGUAL A 7
      
      3 – MAIOR OU IGUAL A 8
      
      8 – NÃO SE APLICA
      
      9 – X	
*   **OUTRACLA**: Outra classificação de estadiamento (char = 20).
*   **META01**: Metástase (char = 3).	Formato: C99
*   **META02**: Metástase (char = 3).	Formato: C99
*   **META03**: Metástase (char = 3).	Formato: C99
*   **META04**: Metástase (char = 3).	Formato: C99
*   **DTTRAT**: Data de inicio do tratamento (date = 10). Formato: DD/MM/YYYY	
*   **NAOTRAT**: Código da razão para não realização do tratamento (int = 1).
      
      1 – RECUSA DO TRATAMENTO
      
      2 – DOENÇA AVANÇADA, FALTA DE CONDIÇÕES CLINICAS
      
      3 – OUTRAS DOENÇAS ASSOCIADAS
      
      4 – ABANDONO DE TRATAMENTO
      
      5 – OBITO POR CANCER
      
      6 – OBITO POR OUTRAS CAUSAS, SOE
      
      7 – OUTRAS
      
      8 – NÃO SE APLICA (CASO TENHA TRATAMENTO)
      
      9 – SEM INFORMAÇÃO	
*   **TRATAMENTO**: Código de combinação dos tratamentos realizados (char = 1).
      
      A – Cirurgia
      
      B – Radioterapia
      
      C – Quimioterapia
      
      D – Cirurgia + Radioterapia
      
      E – Cirurgia + Quimioterapia
      
      F – Radioterapia + Quimioterapia
      
      G – Cirurgia + Radio + Quimio
      
      H – Cirurgia + Radio + Quimio + Hormonio
      
      I – Outras combinações de tratamento
      
      J – Nenhum tratamento realizado	
*   **TRATHOSP**: Código de combinação dos tratamentos realizados no hospital (char = 1).
      
      A – Cirurgia
      
      B – Radioterapia
      
      C – Quimioterapia
      
      D – Cirurgia + Radioterapia
      
      E – Cirurgia + Quimioterapia
      
      F – Radioterapia + Quimioterapia
      
      G – Cirurgia + Radio + Quimio
      
      H – Cirurgia + Radio + Quimio + Hormonio
      
      I – Outras combinações de tratamento

      J – Nenhum tratamento realizado	
*   **TRATFANTES**: Código de combinação dos tratamentos realizados antes/durante admissão fora do hospital (char = 1).
      
      A – Cirurgia
      
      B – Radioterapia
      
      C – Quimioterapia
      
      D – Cirurgia + Radioterapia
      
      E – Cirurgia + Quimioterapia
      
      F – Radioterapia + Quimioterapia
      
      G – Cirurgia + Radio + Quimio
      
      H – Cirurgia + Radio + Quimio + Hormonio
      
      I – Outras combinações de tratamento
      
      J – Nenhum tratamento realizado
      
      K – Sem informação
*   **TRATFAPOS**: Código de combinação dos tratamentos realizados após admissão fora do hospital (char = 1).
      
      A – Cirurgia
      
      B – Radioterapia
      
      C – Quimioterapia
      
      D – Cirurgia + Radioterapia
      
      E – Cirurgia + Quimioterapia
      
      F – Radioterapia + Quimioterapia
      
      G – Cirurgia + Radio + Quimio
      
      H – Cirurgia + Radio + Quimio + Hormonio
      
      I – Outras combinações de tratamento
      
      J – Nenhum tratamento realizado
      
      K – Sem informação
*   **NENHUM**: Tratamento recebido no hospital = nenhum (int = 1). 0 – NÃO; 1 – SIM
*   **CIRURGIA**: Tratamento recebido no hospital = cirurgia (int = 1). 0 – NÃO; 1 – SIM
*   **RADIO**: Tratamento recebido no hospital = radioterapia (int = 1). 0 – NÃO; 1 – SIM
*   **QUIMIO**: Tratamento recebido no hospital = quimioterapia (int = 1). 0 – NÃO; 1 – SIM
*   **HORMONIO**: Tratamento recebido no hospital = hormonioterapia (int = 1). 0 – NÃO; 1 – SIM
*   **TMO**: Tratamento recebido no hospital = tmo (int = 1). 0 – NÃO; 1 – SIM
*   **IMUNO**: Tratamento recebido no hospital = imunoterapia (int = 1). 0 – NÃO; 1 – SIM
*   **OUTROS**: Tratamento recebido no hospital = outros (int = 1). 0 – NÃO; 1 – SIM
*   **NENHUMANT**: Tratamento recebido fora do hospital e antes da admissão = nenhum (int = 1). 0 – NÃO; 1 – SIM
*   **CIRURANT**: Tratamento recebido fora do hospital e antes da admissão = cirurgia (int = 1). 0 – NÃO; 1 – SIM
*   **RADIOANT**: Tratamento recebido fora do hospital e antes da admissão = radioterapia (int = 1). 0 – NÃO; 1 – SIM
*   **QUIMIOANT**: Tratamento recebido fora do hospital e antes da admissão = quimioterapia (int = 1). 0 – NÃO; 1 – SIM
*   **HORMOANT**: Tratamento recebido fora do hospital e antes da admissão = hormonioterapia (int = 1). 0 – NÃO; 1 – SIM
*   **TMOANT**: Tratamento recebido fora do hospital e antes da admissão = tmo (int = 1). 0 – NÃO; 1 – SIM
*   **IMUNOANT**: Tratamento recebido fora do hospital e antes da admissão = imunoterapia (int = 1). 0 – NÃO; 1 – SIM
*   **OUTROANT**: Tratamento recebido fora do hospital e antes da admissão = outros (int = 1). 0 – NÃO; 1 – SIM	
*   **NENHUMAPOS**: Tratamento recebido fora do hospital e durante/após admissão = nenhum	(int = 1). 0 – NÃO; 1 – SIM	
*   **CIRURAPOS**: Tratamento recebido fora do hospital e durante/após admissão = cirurgia	(int = 1). 0 – NÃO; 1 – SIM	
*   **RADIOAPOS**: Tratamento recebido fora do hospital e durante/após admissão = radioterapia	(int = 1). 0 – NÃO; 1 – SIM	
*   **QUIMIOAPOS**: Tratamento recebido fora do hospital e durante/após admissão = quimioterapia	(int = 1). 0 – NÃO; 1 – SIM		
*   **HORMOAPOS**: Tratamento recebido fora do hospital e durante/após admissão = hormonioterapia	(int = 1). 0 – NÃO; 1 – SIM		
*   **TMOAPOS**: Tratamento recebido fora do hospital e durante/após admissão = tmo	(int = 1). 0 – NÃO; 1 – SIM		
*   **IMUNOAPOS**: Tratamento recebido fora do hospital e durante/após admissão = imunoterapia	(int = 1). 0 – NÃO; 1 – SIM	
*   **OUTROAPOS**: Tratamento recebido fora do hospital e durante/após admissão = outros	(int = 1). 0 – NÃO; 1 – SIM	
*   **DTULTINFO**: Data da última informação do paciente (date = 10). Formato: DD/MM/YYYY
*   **ULTINFO**: Última informação sobre o paciente (int = 1).
      
      1 – VIVO, COM CÂNCER
      
      2 – VIVO, SOE
      
      3 – OBITO POR CANCER
      
      4 – OBITO POR OUTRAS CAUSAS, SOE
*   **CONSDIAG**: Diferença em dias entre as datas de consulta o diagnóstico (num = dias).
*   **TRATCONS**: Diferença em dias entre as datas de consulta e tratamento (num = dias).	
*   **DIAGTRAT**: Diferença em dias entre as datas de tratamento e diagnóstico (num = dias).
*   **ANODIAG**: Ano de diagnóstico (int = 4). Formato: 9999
*   **CICI**: Tumor infantil (char = 5).	
*   **CICIGRUP**: Tumor infantil – Grupo (char = 80).	
*   **CICISUBGRU**: Tumor infantil – Sub grupo (char = 80).	
*   **FAIXAETAR**: Faixa etária do paciente (char = 5).	
*   **LATERALI**: Lateralidade (int = 1).
      
      1 – DIREITA
      
      2 – ESQUERDA
      
      3 – BILATERAL
      
      8 - NÃO SE APLICA	
*   **INSTORIG**: Instituição de origem (char = 200). Obrigatório somente se DIAGPREV = 03 – COM DIAGNÓSTICO / COM TRATAMENTO
*   **DRS**: Departamentos Regionais de Saúde (char = 200).
*   **RRAS**: RRAS (char = 200).	
*   **PERDASEG**: Perda de seguimento (int = 1). 
      
      0 – Não
      
      1 – Sim
      
      8 – Não se aplica (excluído do cálculo para o indicador perda de seguimento)	
*   **ERRO**: Admissão com erro (int = 1). 0 – Sem; 1 – Com
*   **DTRECIDIVA**: Data da última ocorrência de recidiva (date = 10). Formato: DD/MM/YYYY	
*   **RECNENHUM**: Sem recidiva (int = 1). 0 - Não; 1 - Sim
*   **RECLOCAL**: Recidiva local (int = 1). 0 - Não; 1 - Sim	
*   **RECREGIO**: Recidiva regional (int = 1). 0 - Não; 1 - Sim	
*   **RECDIST**: Recidiva a distância / metástase (int = 1). 0 - Não; 1 - Sim	
*   **REC01**: Local da recidiva/metástase (char = 3). Formato: C99 
*   **REC02**: Local da recidiva/metástase (char = 3). Formato: C99 	
*   **REC03**: Local da recidiva/metástase (char = 3). Formato: C99 
*   **REC04**: Local da recidiva/metástase (char = 3). Formato: C99 	
*   **IBGEATEN**: Código IBGE da instituição (int = 7).	
*   **CIDO**: Código da morfologia 3ª Edição (int = 5). Formato: 99999	
*   **DSCCIDO**: Descrição da morfologia 3ª Edição (char = 89).




# **Dados**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Trabalho/Cancer/Datasets/cancer_preprocessing.csv')
print(df.shape)
df.head(3)

(943659, 73)


,ESCOLARI,IDADE,SEXO,IBGE,CATEATEND,CLINICA,DIAGPREV,BASEDIAG,TOPO,TOPOGRUP,MORFO,EC,ECGRUP,T,N,M,PT,PN,PM,G,LOCALTNM,IDMITOTIC,PSA,GLEASON,META01,META02,META03,META04,NAOTRAT,TRATAMENTO,TRATHOSP,TRATFANTES,TRATFAPOS,NENHUM,CIRURGIA,RADIO,QUIMIO,HORMONIO,TMO,IMUNO,OUTROS,NENHUMANT,CIRURANT,RADIOANT,NENHUMAPOS,CIRURAPOS,RADIOAPOS,QUIMIOAPOS,HORMOAPOS,TMOAPOS,IMUNOAPOS,OUTROAPOS,ULTINFO,CONSDIAG,TRATCONS,DIAGTRAT,ANODIAG,CICI,CICIGRUP,FAIXAETAR,LATERALI,DRS,RRAS,PERDASEG,RECNENHUM,RECLOCAL,RECREGIO,RECDIST,REC01,REC02,REC03,REC04,IBGEATEN
0,4,40.0,2,3530805,9,15,1,3,222,45,81402,0,0,25,15,0,30,0,0,8,8,8,8,8,0,0,0,0,8,0,0,2,9,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,2,7,95.0,88.0,2000,23,5,4,8,14,15,1,1,0,0,0,0,0,0,0,3509502
1,9,45.0,2,3509502,9,15,1,3,222,45,80703,19,3,25,15,0,41,22,7,8,8,8,8,8,0,0,0,0,8,5,5,2,9,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,3,12,15.0,3.0,2000,23,5,4,8,7,15,0,1,0,0,0,0,0,0,0,3509502
2,2,63.0,2,3509502,9,15,1,3,222,45,80703,19,3,25,15,0,41,22,7,8,8,8,8,8,0,0,0,0,8,1,1,2,9,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,3,6,15.0,9.0,2000,23,5,6,8,7,15,0,1,0,0,0,0,0,0,0,3509502


In [ ]:
df.isna().sum().sort_values(ascending=False).head()

IBGEATEN     0
RADIO        0
G            0
LOCALTNM     0
IDMITOTIC    0
dtype: int64

In [ ]:
col = df.columns
X = df[col].values
X.shape

(943659, 73)

## **Normalização**

In [ ]:
scaler = StandardScaler()
X_norm = scaler.fit_transform(X)

In [ ]:
pd.DataFrame(data=X_norm, columns=col).describe()

,ESCOLARI,IDADE,SEXO,IBGE,CATEATEND,CLINICA,DIAGPREV,BASEDIAG,TOPO,TOPOGRUP,MORFO,EC,ECGRUP,T,N,M,PT,PN,PM,G,LOCALTNM,IDMITOTIC,PSA,GLEASON,META01,META02,META03,META04,NAOTRAT,TRATAMENTO,TRATHOSP,TRATFANTES,TRATFAPOS,NENHUM,CIRURGIA,RADIO,QUIMIO,HORMONIO,TMO,IMUNO,OUTROS,NENHUMANT,CIRURANT,RADIOANT,NENHUMAPOS,CIRURAPOS,RADIOAPOS,QUIMIOAPOS,HORMOAPOS,TMOAPOS,IMUNOAPOS,OUTROAPOS,ULTINFO,CONSDIAG,TRATCONS,DIAGTRAT,ANODIAG,CICI,CICIGRUP,FAIXAETAR,LATERALI,DRS,RRAS,PERDASEG,RECNENHUM,RECLOCAL,RECREGIO,RECDIST,REC01,REC02,REC03,REC04,IBGEATEN
count,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05,9.436590e+05
mean,-1.130038e-13,-1.080222e-14,-1.537774e-13,1.360224e-15,6.779968e-13,-7.516858e-16,-7.868857e-14,-5.306860e-14,-5.120349e-14,6.222969e-14,3.868816e-14,-3.518428e-14,1.156337e-13,-1.236059e-14,-3.104793e-13,1.803285e-13,1.593043e-13,-4.103677e-13,-2.304149e-13,-6.434715e-13,8.263831e-14,-9.542169e-15,-3.035037e-13,-1.493150e-14,1.421304e-13,2.221803e-13,-2.244972e-13,-2.791250e-13,-4.523499e-13,-2.713785e-14,-2.710381e-14,-1.347933e-14,9.824199e-14,9.139386e-14,-1.864898e-13,-2.963625e-14,3.033464e-14,1.809813e-13,9.093163e-14,1.771232e-13,7.649916e-14,2.100906e-13,-8.255507e-15,-8.255748e-15,2.040650e-13,-1.885738e-13,-9.541332e-14,-1.642661e-13,5.783170e-15,-2.095035e-14,-4.271488e-14,5.884254e-14,-5.514632e-13,3.696699e-14,7.931577e-15,2.190875e-14,-4.831541e-14,-2.353950e-14,1.675945e-13,3.523164e-13,1.251109e-13,-4.931823e-13,-1.359398e-13,-6.673524e-13,1.427914e-13,3.468548e-13,-6.341165e-13,-8.040598e-13,-2.720097e-13,-3.067903e-14,3.201143e-13,1.166343e-13,-5.947767e-13
std,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,-1.146847e+00,-3.546734e+00,-1.000733e+00,-6.635249e+00,-1.108539e+00,-1.621715e+00,-7.686313e-01,-8.091746e+00,-2.618457e+00,-2.590543e+00,-7.943764e-01,-1.467136e+00,-1.519387e+00,-1.282202e+00,-7.610688e-01,-5.246297e-01,-2.170617e+00,-1.747470e+00,-1.739749e+00,-7.190407e+00,-1.967641e+01,-5.199395e+01,-6.047915e+00,-6.262922e+00,-3.572571e-01,-1.894448e-01,-1.055113e-01,-5.373153e-02,-7.590048e+00,-1.011710e+00,-9.935248e-01,-8.688649e+02,-5.995664e+00,-3.015171e-01,-1.288288e+00,-5.928398e-01,-7.343145e-01,-3.728359e-01,-6.248441e-02,-8.

#**GMM**

In [ ]:
ncluster = 21

gmm = GaussianMixture(n_components =  ncluster, init_params='random', random_state=7)
gmm.fit(X_norm)

GaussianMixture(covariance_type='full', init_params='random', max_iter=100,
                means_init=None, n_components=21, n_init=1,
                precisions_init=None, random_state=7, reg_covar=1e-06,
                tol=0.001, verbose=0, verbose_interval=10, warm_start=False,
                weights_init=None)

In [ ]:
y_cluster = gmm.predict(X_norm)
y_cluster

array([6, 6, 6, ..., 2, 6, 6])

In [ ]:
df['GRUPO'] = y_cluster
df.head()

,ESCOLARI,IDADE,SEXO,IBGE,CATEATEND,CLINICA,DIAGPREV,BASEDIAG,TOPO,TOPOGRUP,MORFO,EC,ECGRUP,T,N,M,PT,PN,PM,G,LOCALTNM,IDMITOTIC,PSA,GLEASON,META01,META02,META03,META04,NAOTRAT,TRATAMENTO,TRATHOSP,TRATFANTES,TRATFAPOS,NENHUM,CIRURGIA,RADIO,QUIMIO,HORMONIO,TMO,IMUNO,OUTROS,NENHUMANT,CIRURANT,RADIOANT,NENHUMAPOS,CIRURAPOS,RADIOAPOS,QUIMIOAPOS,HORMOAPOS,TMOAPOS,IMUNOAPOS,OUTROAPOS,ULTINFO,CONSDIAG,TRATCONS,DIAGTRAT,ANODIAG,CICI,CICIGRUP,FAIXAETAR,LATERALI,DRS,RRAS,PERDASEG,RECNENHUM,RECLOCAL,RECREGIO,RECDIST,REC01,REC02,REC03,REC04,IBGEATEN,GRUPO
0,4,40.0,2,3530805,9,15,1,3,222,45,81402,0,0,25,15,0,30,0,0,8,8,8,8,8,0,0,0,0,8,0,0,2,9,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,2,7,95.0,88.0,2000,23,5,4,8,14,15,1,1,0,0,0,0,0,0,0,3509502,6
1,9,45.0,2,3509502,9,15,1,3,222,45,80703,19,3,25,15,0,41,22,7,8,8,8,8,8,0,0,0,0,8,5,5,2,9,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,3,12,15.0,3.0,2000,23,5,4,8,7,15,0,1,0,0,0,0,0,0,0,3509502,6
2,2,63.0,2,3509502,9,15,1,3,222,45,80703,19,3,25,15,0,41,22,7,8,8,8,8,8,0,0,0,0,8,1,1,2,9,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,3,6,15.0,9.0,2000,23,5,6,8,7,15,0,1,0,0,0,0,0,0,0,3509502,6
3,9,64.0,2,3545803,9,15,1,3,222,45,80703,19,3,25,15,0,41,22,7,8,8,8,8,8,0,0,0,0,8,1,1,2,9,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,3,6,33.0,27.0,2000,23,5,6,8,7,15,0,1,0,0,0,0,0,0,0,3509502,6
4,1,48.0,2,3530805,9,15,2,3,222,45,80703,19,3,25,15,0,41,22,7,8,8,8,8,8,0,0,0,0,8,1,1,2,9,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,2,0,132.0,132.0,2000,23,5,4,8,14,15,1,1,0,0,0,0,0,0,0,3509502,6


## **Salvando os dados**

In [ ]:
df.to_csv('/content/drive/MyDrive/Trabalho/Cancer/Datasets/gmm_preprocessing.csv', encoding='utf-8', index=False)

## **Salvando modelo e StandardScaler**

In [ ]:
with open('/content/drive/MyDrive/Trabalho/Cancer/Modelos/gmm_scaler.pkl', 'wb') as handle:
    pickle.dump({'scaler': scaler}, handle)

In [ ]:
with open('/content/drive/MyDrive/Trabalho/Cancer/Modelos/gmm.pkl', 'wb') as handle:
    pickle.dump({'gmm': gmm}, handle)

# **Subset**

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Trabalho/Cancer/Datasets/gmm_preprocessing.csv')
print(df.shape)
df.head(3)

(943659, 74)


,ESCOLARI,IDADE,SEXO,IBGE,CATEATEND,CLINICA,DIAGPREV,BASEDIAG,TOPO,TOPOGRUP,MORFO,EC,ECGRUP,T,N,M,PT,PN,PM,G,LOCALTNM,IDMITOTIC,PSA,GLEASON,META01,META02,META03,META04,NAOTRAT,TRATAMENTO,TRATHOSP,TRATFANTES,TRATFAPOS,NENHUM,CIRURGIA,RADIO,QUIMIO,HORMONIO,TMO,IMUNO,OUTROS,NENHUMANT,CIRURANT,RADIOANT,NENHUMAPOS,CIRURAPOS,RADIOAPOS,QUIMIOAPOS,HORMOAPOS,TMOAPOS,IMUNOAPOS,OUTROAPOS,ULTINFO,CONSDIAG,TRATCONS,DIAGTRAT,ANODIAG,CICI,CICIGRUP,FAIXAETAR,LATERALI,DRS,RRAS,PERDASEG,RECNENHUM,RECLOCAL,RECREGIO,RECDIST,REC01,REC02,REC03,REC04,IBGEATEN,GRUPO
0,4,40.0,2,3530805,9,15,1,3,222,45,81402,0,0,25,15,0,30,0,0,8,8,8,8,8,0,0,0,0,8,0,0,2,9,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,2,7,95.0,88.0,2000,23,5,4,8,14,15,1,1,0,0,0,0,0,0,0,3509502,6
1,9,45.0,2,3509502,9,15,1,3,222,45,80703,19,3,25,15,0,41,22,7,8,8,8,8,8,0,0,0,0,8,5,5,2,9,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,3,12,15.0,3.0,2000,23,5,4,8,7,15,0,1,0,0,0,0,0,0,0,3509502,6
2,2,63.0,2,3509502,9,15,1,3,222,45,80703,19,3,25,15,0,41,22,7,8,8,8,8,8,0,0,0,0,8,1,1,2,9,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,3,6,15.0,9.0,2000,23,5,6,8,7,15,0,1,0,0,0,0,0,0,0,3509502,6


In [4]:
df.GRUPO.value_counts()

6     566893
2      86284
17     77045
7      51663
13     33575
18     28417
19     19429
12     16131
8      14058
5       8589
14      6631
20      6593
11      6332
10      5725
0       4854
3       3843
16      2639
4       2036
9       1539
15       855
1        528
Name: GRUPO, dtype: int64

In [5]:
df.sample(n = 5000, random_state=7).sort_index()

,ESCOLARI,IDADE,SEXO,IBGE,CATEATEND,CLINICA,DIAGPREV,BASEDIAG,TOPO,TOPOGRUP,MORFO,EC,ECGRUP,T,N,M,PT,PN,PM,G,LOCALTNM,IDMITOTIC,PSA,GLEASON,META01,META02,META03,META04,NAOTRAT,TRATAMENTO,TRATHOSP,TRATFANTES,TRATFAPOS,NENHUM,CIRURGIA,RADIO,QUIMIO,HORMONIO,TMO,IMUNO,OUTROS,NENHUMANT,CIRURANT,RADIOANT,NENHUMAPOS,CIRURAPOS,RADIOAPOS,QUIMIOAPOS,HORMOAPOS,TMOAPOS,IMUNOAPOS,OUTROAPOS,ULTINFO,CONSDIAG,TRATCONS,DIAGTRAT,ANODIAG,CICI,CICIGRUP,FAIXAETAR,LATERALI,DRS,RRAS,PERDASEG,RECNENHUM,RECLOCAL,RECREGIO,RECDIST,REC01,REC02,REC03,REC04,IBGEATEN,GRUPO
47,2,80.0,2,3514908,9,33,1,3,212,42,85003,3,1,1,0,0,10,0,0,8,8,8,8,8,0,0,0,0,8,0,0,2,9,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,2,21,75.0,54.0,2000,23,5,7,8,10,14,1,1,0,0,0,0,0,0,0,3509502,6
113,9,46.0,2,3536505,9,33,2,3,212,42,85003,19,3,16,1,0,22,1,0,8,8,8,8,8,0,0,0,0,8,7,7,2,9,0,1,1,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,4,0,44.0,44.0,2000,23,5,4,8,7,15,0,0,0,1,0,37,0,0,0,3509502,7
176,9,57.0,2,3108909,9,33,2,3,212,42,85003,12,2,6,0,0,1,0,0,8,8,8,8,8,0,0,0,0,8,8,8,2,9,0,1,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,2,0,29.0,29.0,2000,23,5,5,8,0,99,1,1,0,0,0,0,0,0,0,3509502,6
232,9,80.0,2,3509502,2,33,1,3,212,42,85203,12,2,6,0,0,1,0,0,8,8,8,8,8,0,0,0,0,8,8,8,2,9,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,4,11,63.0,52.0,2000,23,5,7,2,7,15,0,1,0,0,0,0,0,0,0,3509502,6
279,9,62.0,2,3159605,9,15,2,3,222,45,80703,15,2,25,15,0,41,22,7,8,8,8,8,8,0,0,0,0,8,1,1,2,9,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,3,0,23.0,23.0,2000,23,5,6,8,0,99,0,1,0,0,0,0,0,0,0,3509502,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942624,4,48.0,1,3159803,2,24,2,3,245,52,85503,15,2,9,0,0,33,21,9,8,8,8,1,2,0,0,0,0,8,0,0,2,9,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,129,125.0,254.0,2018,23,5,4,8,0,99,0,1,0,0,0,0,0,0,0,3524808,18
942887,2,42.0,2,3515509,2,24,2,3,25,5,80703,17,3,10,0,0,33,21,9,8,8,8,8,8,0,0,0,0,8,5,5,2,9,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,53,72.0,125.0,2019,23,5,4,8,15,12,0,1,0,0,0,0,0,0,0,3524808,6
942994,2,68.0,2,3134400,2,24,2,3,176,38,80903,3,1,1,0,0,33,21,9,8,8,8,8,8,0,0,0,0,7,9,9,2,9,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,2,448,39.0,32.0,2018,23,5,6,8,0,99,0,1,0,0,0,0,0,0,0,3524808,2
943077,5,60.0,1,3515509,2,24,2,3,330,69,81403,24,4,25,15,1,33,21,9,8,8,8,8,8,68,0,0,0,2,9,9,2,9,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,2,40,39.0,32.0,2019,23,5,6,8,15,12,0,1,0,0,0,0,0,0,0,3524808,2


In [6]:
n_samples = 10000 # samples from dataset

df_subset = df.sample(n_samples, random_state=7).sort_index().copy()
df_subset.GRUPO.value_counts()

6     5858
2      924
17     891
7      580
13     355
18     323
19     215
12     153
8      148
5       93
20      84
11      83
14      65
10      53
0       51
3       41
16      31
4       21
9       14
15      10
1        7
Name: GRUPO, dtype: int64

In [7]:
cols = df.columns
cols = cols.drop(['GRUPO'])
len(cols)

73

In [8]:
df_subset[cols].shape

(10000, 73)

## **Normalização**



In [9]:
X = df_subset[cols].values
X.shape

(10000, 73)

In [10]:
scaler = StandardScaler()
X_norm = scaler.fit_transform(X)

In [11]:
pd.DataFrame(data=X_norm, columns=cols).describe()

,ESCOLARI,IDADE,SEXO,IBGE,CATEATEND,CLINICA,DIAGPREV,BASEDIAG,TOPO,TOPOGRUP,MORFO,EC,ECGRUP,T,N,M,PT,PN,PM,G,LOCALTNM,IDMITOTIC,PSA,GLEASON,META01,META02,META03,META04,NAOTRAT,TRATAMENTO,TRATHOSP,TRATFANTES,TRATFAPOS,NENHUM,CIRURGIA,RADIO,QUIMIO,HORMONIO,TMO,IMUNO,OUTROS,NENHUMANT,CIRURANT,RADIOANT,NENHUMAPOS,CIRURAPOS,RADIOAPOS,QUIMIOAPOS,HORMOAPOS,TMOAPOS,IMUNOAPOS,OUTROAPOS,ULTINFO,CONSDIAG,TRATCONS,DIAGTRAT,ANODIAG,CICI,CICIGRUP,FAIXAETAR,LATERALI,DRS,RRAS,PERDASEG,RECNENHUM,RECLOCAL,RECREGIO,RECDIST,REC01,REC02,REC03,REC04,IBGEATEN
count,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,10000.0,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,10000.0,10000.0,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04
mean,-8.859358e-16,8.159029e-16,1.181544e-15,7.986162e-16,9.289391e-15,1.514527e-15,1.353451e-15,5.957457e-16,-4.098541e-16,-6.483439e-16,-2.823830e-15,1.645439e-15,5.773382e-16,3.623235e-15,1.709188e-15,-1.274536e-15,-5.033829e-15,2.237610e-15,6.889600e-15,-3.738460e-15,-5.035861e-16,-1.247846e-15,4.381501e-15,-8.059226e-15,2.427059e-16,-2.996808e-15,6.332518e-16,-1.884272e-15,1.011347e-15,3.197886e-16,-2.236877e-16,0.0,-1.535000e-15,-3.478018e-15,-3.352696e-15,1.726685e-15,1.437095e-15,-1.058920e-15,-1.457365e-15,2.189990e-15,-6.115242e-15,-3.262036e-16,0.0,0.0,2.898137e-15,1.484401e-15,-3.244571e-16,-3.158251e-16,3.606962e-16,-1.523347e-16,1.971477e-15,-3.160716e-15,-3.462941e-15,2.861406e-16,-5.567213e-17,-7.970624e-16,-1.346643e-14,-9.750534e-17,-1.205075e-15,1.680722e-15,-6.789014e-17,1.403575e-14,1.730444e-15,1.952074e-14,7.771428e-15,-7.851914e-15,2.155442e-16,1.363338e-14,-3.794182e-15,-2.505463e-15,-3.549522e-16,5.807591e-16,1.125388e-14
std,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,0.0,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,0.0,0.0,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00
min,-1.133870e+00,-3.550412e+00,-1.005415e+00,-6.788184e+00,-1.109469e+00,-1.613128e+00,-7.793963e-01,-7.546369e+00,-2.631340e+00,-2.601522e+00,-7.947671e-01,-1.479956e+00,-1.525293e+00,-1.297038e+00,-7.714408e-01,-5.342507e-01,-2.180944e+00,-1.760598e+00,-1.769725e+00,-7.127178e+00,-1.755974e+01,-4.864311e+01,-5.792416e+00,-5.990581e+00,-3.589617e-01,-1.852981e-01,-1.102052e-01,-5.402606e-02,-7.612654e+00,-1.021117e+00,-9.999033e-01,0.0,-5.970787e+00,-3.075239e-01,-1.278146e+00,-5.862805e-01,-7.464230e-01,-3.713695e-01,-6.797983e-02,-7.834180e-02,-2.533168e-01,-3.778322e+01,0.0,0.0,-3.986464e+00,-7.229925e-02,-1.755607e-

# **PCA e TSNE**

In [12]:
pca = PCA(n_components=2)
pca_result = pca.fit_transform(X_norm)

df_subset['pca_one'] = pca_result[:,0]
df_subset['pca_two'] = pca_result[:,1]

print('Cumulative explained variation for 2 principal components: {}'.format(np.sum(pca.explained_variance_ratio_)))

Cumulative explained variation for 2 principal components: 0.13996959038827225


In [13]:
fig = px.scatter(df_subset, x="pca_one", y="pca_two", color=df_subset.GRUPO.astype(str),
                 height=700, width=1000
                )
fig.update_traces(marker=dict(size=7,
                              line=dict(width=0.7, 
                                        color='white')),
                  selector=dict(mode='markers'))

fig.show()

In [14]:
pca_50 = PCA(n_components=50)

pca_result_50 = pca_50.fit_transform(X_norm)

print('Cumulative explained variation for 50 principal components: {}'.format(np.sum(pca_50.explained_variance_ratio_)))

Cumulative explained variation for 50 principal components: 0.9560360592547131


In [15]:
tsne = TSNE(n_components=2, random_state=7, perplexity=30)
tsne_pca_results = tsne.fit_transform(pca_result_50)

df_subset['tsne_pca50_one'] = tsne_pca_results[:,0]
df_subset['tsne_pca50_two'] = tsne_pca_results[:,1]

In [16]:
fig = px.scatter(df_subset, x="tsne_pca50_one", y="tsne_pca50_two", color=df_subset.GRUPO.astype(str),
                 height=700, width = 1000
                )
fig.update_traces(marker=dict(size=7,
                              line=dict(width=0.7, color='white'
                              )),
                  selector=dict(mode='markers'))

fig.show()

In [17]:
tsne = TSNE(n_components=2, random_state=7, perplexity=50)
tsne_pca_results = tsne.fit_transform(pca_result_50)

df_subset['tsne_pca50_one'] = tsne_pca_results[:,0]
df_subset['tsne_pca50_two'] = tsne_pca_results[:,1]

In [18]:
fig = px.scatter(df_subset, x="tsne_pca50_one", y="tsne_pca50_two", color=df_subset.GRUPO.astype(str),
                 height=700, width = 1000
                )
fig.update_traces(marker=dict(size=7,
                              line=dict(width=0.7, color='white'
                              )),
                  selector=dict(mode='markers'))

fig.show()

In [19]:
df_gmm = pd.DataFrame()
df_gmm['tsne_pca50_one'] = df_subset['tsne_pca50_one'].copy()
df_gmm['tsne_pca50_two'] = df_subset['tsne_pca50_two'].copy()
df_gmm['grupo_gmm'] = df_subset['GRUPO'].copy()
df_gmm.reset_index(inplace=True)
df_gmm.head()

,index,tsne_pca50_one,tsne_pca50_two,grupo_gmm
0,47,32.933277,-8.992918,6
1,84,-10.522817,-22.701649,6
2,113,-19.524906,64.889526,7
3,176,16.489996,22.443138,6
4,232,20.858507,33.863190,6


In [20]:
df_gmm.to_csv('/content/drive/MyDrive/Trabalho/Cancer/Datasets/gmm_tsne.csv', encoding='utf-8', index=False)

In [21]:
tsne = TSNE(n_components=2, random_state=7, perplexity=100)
tsne_pca_results = tsne.fit_transform(pca_result_50)

df_subset['tsne_pca50_one'] = tsne_pca_results[:,0]
df_subset['tsne_pca50_two'] = tsne_pca_results[:,1]

In [22]:
fig = px.scatter(df_subset, x='tsne_pca50_one', y='tsne_pca50_two', color=df_subset.GRUPO.astype(str),
                 height=700, width = 1000
                )
fig.update_traces(marker=dict(size=7,
                              line=dict(width=0.7, color='white'
                              )),
                  selector=dict(mode='markers'))

fig.show()

# **Balanceado**

In [23]:
df_test = df[df.GRUPO == 1].copy()
n = df_test.shape[0]
for i in range(21):
  if i != 1:
    df_aux = df[df.GRUPO == i].sample(n, random_state=7).copy()
    df_test = pd.concat([df_test, df_aux])

df_test.GRUPO.value_counts().sort_index()

0     528
1     528
2     528
3     528
4     528
5     528
6     528
7     528
8     528
9     528
10    528
11    528
12    528
13    528
14    528
15    528
16    528
17    528
18    528
19    528
20    528
Name: GRUPO, dtype: int64

In [24]:
cols = df_test.columns
cols = cols.drop(['GRUPO'])
len(cols)

73

In [25]:
X = df_test[cols].values
X.shape

(11088, 73)

In [26]:
s = StandardScaler()
X_norm = s.fit_transform(X)

In [27]:
pd.DataFrame(data=X_norm, columns=cols).describe()

,ESCOLARI,IDADE,SEXO,IBGE,CATEATEND,CLINICA,DIAGPREV,BASEDIAG,TOPO,TOPOGRUP,MORFO,EC,ECGRUP,T,N,M,PT,PN,PM,G,LOCALTNM,IDMITOTIC,PSA,GLEASON,META01,META02,META03,META04,NAOTRAT,TRATAMENTO,TRATHOSP,TRATFANTES,TRATFAPOS,NENHUM,CIRURGIA,RADIO,QUIMIO,HORMONIO,TMO,IMUNO,OUTROS,NENHUMANT,CIRURANT,RADIOANT,NENHUMAPOS,CIRURAPOS,RADIOAPOS,QUIMIOAPOS,HORMOAPOS,TMOAPOS,IMUNOAPOS,OUTROAPOS,ULTINFO,CONSDIAG,TRATCONS,DIAGTRAT,ANODIAG,CICI,CICIGRUP,FAIXAETAR,LATERALI,DRS,RRAS,PERDASEG,RECNENHUM,RECLOCAL,RECREGIO,RECDIST,REC01,REC02,REC03,REC04,IBGEATEN
count,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,11088.0,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,11088.0,11088.0,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04,1.108800e+04
mean,-8.162963e-17,-2.256718e-16,3.564769e-16,5.210967e-16,-1.263219e-16,-2.773555e-18,-8.129840e-15,1.082734e-15,-3.737961e-15,-2.420933e-15,-4.949234e-15,-1.064705e-15,-1.661690e-14,2.437044e-15,-2.437905e-15,-1.622622e-14,1.594894e-15,-3.720959e-15,4.001148e-15,3.566381e-15,3.427265e-15,-3.650698e-14,1.810260e-14,2.055883e-14,2.825902e-15,-3.269891e-15,-3.246281e-15,-6.517586e-15,3.098251e-15,-1.872740e-15,2.566109e-15,0.0,1.347472e-15,-4.467406e-15,-4.683823e-15,-3.480521e-15,-6.418247e-15,-5.317666e-15,6.485127e-15,-3.235957e-14,-8.338538e-16,-7.141193e-15,0.0,0.0,-8.506033e-15,-6.913356e-15,-2.877163e-15,9.455835e-15,-1.101160e-14,1.317412e-14,5.864399e-15,9.649478e-15,2.464399e-15,1.142424e-15,1.921137e-16,4.246343e-16,3.801302e-15,3.130305e-15,3.624017e-15,-1.015857e-15,7.617664e-16,-6.393395e-16,-9.400599e-16,1.011536e-14,-1.392765e-15,-2.480299e-15,-9.268169e-15,1.544516e-14,4.129713e-15,-3.931629e-15,-8.988896e-15,1.530556e-14,-1.388128e-14
std,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,0.0,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,0.0,0.0,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00,1.000045e+00
min,-1.226002e+00,-2.941891e+00,-9.433412e-01,-4.950616e+00,-9.488047e-01,-2.030186e+00,-8.975350e-01,-2.794406e+00,-2.639321e+00,-2.596129e+00,-1.001834e+00,-2.004881e+00,-1.888273e+00,-1.576918e+00,-9.033462e-01,-6.911069e-01,-2.653231e+00,-2.064952e+00,-2.015145e+00,-4.443462e+00,-1.086532e+01,-5.547906e+00,-4.217472e+00,-4.355791e+00,-4.737693e-01,-3.277723e-01,-2.187353e-01,-1.174868e-01,-7.030796e+00,-1.512043e+00,-1.396323e+00,0.0,-2.731306e+00,-3.645976e-01,-9.669946e-01,-5.724903e-01,-9.548425e-01,-4.408093e-01,-1.914398e-01,-2.754599e-01,-2.945152e-01,-9.110434e+00,0.0,0.0,-1.565561e+00,-2.404442e-01,-3.3085

In [28]:
pca = PCA(n_components=2)
pca_result = pca.fit_transform(X_norm)

df_test['pca_one'] = pca_result[:,0]
df_test['pca_two'] = pca_result[:,1]

print('Cumulative explained variation for 2 principal components: {}'.format(np.sum(pca.explained_variance_ratio_)))

Cumulative explained variation for 2 principal components: 0.1540112689037328


In [29]:
fig = px.scatter(df_test, x="pca_one", y="pca_two", color=df_test.GRUPO.astype(str),
                 height=700, width=1000
                )
fig.update_traces(marker=dict(size=7,
                              line=dict(width=0.7, 
                                        color='white')),
                  selector=dict(mode='markers'))

fig.show()

In [30]:
pca_50 = PCA(n_components=50)

pca_result_50 = pca_50.fit_transform(X_norm)

print('Cumulative explained variation for 50 principal components: {}'.format(np.sum(pca_50.explained_variance_ratio_)))

Cumulative explained variation for 50 principal components: 0.9541242918726611


In [31]:
tsne = TSNE(n_components=2, random_state=7, perplexity=50)
tsne_pca_results = tsne.fit_transform(pca_result_50)

df_test['tsne_pca50_one'] = tsne_pca_results[:,0]
df_test['tsne_pca50_two'] = tsne_pca_results[:,1]

In [32]:
fig = px.scatter(df_test, x="tsne_pca50_one", y="tsne_pca50_two", color=df_test.GRUPO.astype(str),
                 height=700, width = 1000
                )
fig.update_traces(marker=dict(size=7,
                              line=dict(width=0.7, color='white'
                              )),
                  selector=dict(mode='markers'))

fig.show()

# **Referências**

https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html

https://medium.com/@violante.andre/an-introduction-to-t-sne-with-python-example-47e6ae7dc58f

https://towardsdatascience.com/t-sne-python-example-1ded9953f26

https://towardsdatascience.com/visualising-high-dimensional-datasets-using-pca-and-t-sne-in-python-8ef87e7915b

https://distill.pub/2016/misread-tsne/